In [31]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow import keras
from keras.layers import Dense,MaxPooling1D,InputLayer,Dropout
from keras.models import Sequential

In [32]:
data=pd.read_csv('Data.csv')
data.drop_duplicates(['RANK'],inplace=True)
# Combine 'BRANCH' and 'COLLEGE' columns into 'target'
data['target'] = data['COLLEGE'] + '_' + data['BRANCH']

# Drop 'CANDIDATE NAME' column
data.drop(['CANDIDATE NAME','COLLEGE','BRANCH',], axis=1, inplace=True)

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
le=LabelEncoder()
data['target']=le.fit_transform(data['target'])
#columns_to_encode = ['GENDER', 'REGION', 'CATEGORY', 'SEAT CATEGORY']
columns_to_encode = ['SEAT CATEGORY','GENDER', 'REGION', 'CATEGORY']

print(data.head())
# Define the columns to be encoded and the columns to be skipped
transfor = Pipeline([
    ('encoding', ColumnTransformer([('one_hot_encoding', OneHotEncoder(), columns_to_encode)], remainder='passthrough'))
])

data_trans=transfor.fit_transform(data.drop('target',axis=1))

   RANK GENDER REGION CATEGORY SEAT CATEGORY  target
0   144      M     OU     BC_B     OC_GEN_UR     534
1   233      M     OU     BC_D     OC_GEN_UR     534
2   243      M     OU     BC_B   BC_B_GEN_UR     534
3   286      M     OU       OC    EWS_GEN_UR     534
4   318      M     OU     BC_D   BC_D_GEN_OU     291


In [34]:
print(type(data_trans))
data_trans=data_trans.toarray()
type(data_trans)

<class 'scipy.sparse._csr.csr_matrix'>


numpy.ndarray

In [97]:
model=Sequential([
    InputLayer(input_shape=(211,)),
    Dense(units=512,activation='relu'),

    Dropout(0.2),
    Dense(units=256,activation='relu'),

    Dropout(0.2),
    Dense(units=128,activation='relu'),

    Dropout(0.2),
    Dense(units=256,activation='relu'),

    Dropout(0.2),
    Dense(units=512,activation='relu'),
    
    Dropout(0.2),
    Dense(units=799,activation='softmax'),
])

In [98]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_97 (Dense)            (None, 512)               108544    
                                                                 
 dropout_80 (Dropout)        (None, 512)               0         
                                                                 
 dense_98 (Dense)            (None, 256)               131328    
                                                                 
 dropout_81 (Dropout)        (None, 256)               0         
                                                                 
 dense_99 (Dense)            (None, 128)               32896     
                                                                 
 dropout_82 (Dropout)        (None, 128)               0         
                                                                 
 dense_100 (Dense)           (None, 256)             

In [101]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [102]:
model.fit(data_trans,data['target'],epochs=10)

Epoch 1/10
1541/1541 [==============================] - 20s 12ms/step - loss: 14.9268 - accuracy: 0.0061
Epoch 2/10
1541/1541 [==============================] - 19s 12ms/step - loss: 6.3569 - accuracy: 0.0061
Epoch 3/10
1541/1541 [==============================] - 19s 13ms/step - loss: 6.3409 - accuracy: 0.0064
Epoch 4/10
1541/1541 [==============================] - 20s 13ms/step - loss: 6.3544 - accuracy: 0.0063
Epoch 5/10
1541/1541 [==============================] - 18s 12ms/step - loss: 6.3376 - accuracy: 0.0065
Epoch 6/10
1541/1541 [==============================] - 19s 12ms/step - loss: 6.3335 - accuracy: 0.0066
Epoch 7/10
1541/1541 [==============================] - 21s 13ms/step - loss: 6.3303 - accuracy: 0.0062
Epoch 8/10
1541/1541 [==============================] - 20s 13ms/step - loss: 6.3367 - accuracy: 0.0065
Epoch 9/10
1541/1541 [==============================] - 19s 13ms/step - loss: 6.3328 - accuracy: 0.0061
Epoch 10/10
1541/1541 [==============================] - 22s 14

In [111]:
t=None
t=[[22791,'F','OU','OC','OC_GIRLS_OU']]  #20k rank, girl
t=pd.DataFrame(t,columns=['RANK','GENDER','REGION','CATEGORY','SEAT CATEGORY'])
tt=None
tt=transfor.transform(t)
tt=tt[0].toarray()
print(np.shape(tt[0]))
print(le.inverse_transform( [np.argmax(model.predict(tt))]) )

(211,)
1/1 [==============================] - 0s 32ms/step
['KMIT_CSE']


In [108]:
model.predict()